In [177]:
import os
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt  
import matplotlib.patches as patches

from datetime import date

from glob import glob
import shutil

In [2]:
mother_path = 'D:/Multi-modal project/analysis/'
#save_path = mother_path+'result/5. Single condition selectivity/'
fig_path = mother_path+'result/figures/'

### Parameter setting

In [65]:
sig_alpha = 0.01
sig_cohend = 0.4
cons_bin_crit = 5

gauss_sigma = 2
gauss_on = True

only_PER = True   # analyze only PER neurons?

condition = ['Multimodal','Visual','Auditory','Control']
cat = ['M','V','A','MV','MA','VA','MVA','None']

# colors for multimodal, vis-only, aud-only conditions
#colorpalette = ['mediumorchid','cornflowerblue','lightcoral','gray']
colorpalette = ['purple','blue','red','gray']

today = str(date.today())

### Data preparation

In [92]:
cell_path = mother_path+'result/zFR export/13-Apr-2022 (5 trials)/'

data_path = mother_path+'result/3. Item-selectivity/2023-01-09/'
df = pd.read_csv(data_path+'2023-01-09_item-selectivity.csv')

# analyze only PER neurons
if only_PER == True:
    df = df[df['Region']=='PER']
    df.reset_index(inplace=True,drop=True)
df.insert(4,'Cat','')

for i in range(len(df)):
    cat_info = ''
    if df.loc[i,'M_sig'] == 1:
        cat_info += 'M'
    if df.loc[i,'V_sig'] == 1:
        cat_info += 'V'
    if df.loc[i,'A_sig'] == 1:
        cat_info += 'A'
    if cat_info=='':
        df.loc[i,'Cat'] = 'None'
    else:
        df.loc[i,'Cat'] = cat_info
df = df[df['Cat']!='None'].reset_index(drop=True)

res_data_path = mother_path+'result/3.1. Response coding/2023-01-16/'
res_df = pd.read_csv(res_data_path+'2023-01-16_response_corr.csv')

response_cell = res_df.loc[res_df.Response_corr>0.5,'Key'].values
# remove response-selective cells
df.drop(df[df['Key'].isin(response_cell)].index,inplace=True)        

### Session-by-session cell categories

In [111]:
result = []

rat_list = df['RatID'].unique()
for rat in rat_list:
    rat_ss = df.loc[df['RatID']==rat,'Session'].unique()
    for ss in rat_ss:
        ss_df = df[(df['RatID']==rat)&(df['Session']==ss)]
        ss_result = {'Rat':rat,
                     'Session':ss,
                     'M':len(ss_df[ss_df['Cat']=='M']),
                     'V':len(ss_df[ss_df['Cat']=='V']),
                     'A':len(ss_df[ss_df['Cat']=='A']),
                     'MV':len(ss_df[ss_df['Cat']=='MV']),
                     'MA':len(ss_df[ss_df['Cat']=='MA']),
                     'VA':len(ss_df[ss_df['Cat']=='VA']),
                     'MVA':len(ss_df[ss_df['Cat']=='MVA'])
                    }
        result.append(ss_result)                

In [94]:
result = pd.DataFrame(result)
result

,Rat,Session,M,V,A,MV,MA,VA,MVA
0,600,1,5,0,0,0,1,0,2
1,600,2,4,0,2,1,3,0,0
2,600,3,3,0,1,0,0,0,0
3,600,4,4,0,1,1,3,1,1
4,600,5,2,0,0,1,2,0,1
5,602,1,3,4,4,2,5,0,1
6,602,2,3,2,4,0,4,1,0
7,602,3,2,3,3,3,1,1,2
8,602,4,4,2,1,0,1,0,1
9,602,5,7,1,0,1,0,2,0


### Cell finder
use it if you want to see object-selective patterns of individual neurons from a specific session

In [212]:
def cell_finder(rat,ss):
    data = df[(df['RatID']==rat)&(df['Session']==ss)]
    return data.Key.values

In [222]:
# Designate target rat and session
rat = 699
ss = 1
target = cell_finder(rat,ss)

In [228]:
cell_path = mother_path+'result/3. Item-selectivity/2023-01-09/PER'
save_path = 'C:/Users/Lim HY/Desktop/cells/'

os.chdir(cell_path)
for cell_run in target:
    if cell_run < 10:
        cell_num = '000'+str(cell_run)
    elif cell_run < 100:
        cell_num = '00'+str(cell_run)
    elif cell_run < 1000:
        cell_num = '0'+str(cell_run)
    else:
        cell_num = str(cell_run)
        
    cell_full_name = glob(cell_num+'*')
    shutil.copyfile(cell_full_name[0],save_path+cell_full_name[0])
    
print('Completed!')

Completed!


### Ensemble firing patterns of a specific session to different conditions

In [242]:
%%time

zfr_path = mother_path+'result/zFR export/13-Apr-2022 (5 trials)/'

rat_list = df['RatID'].unique()
for rat in rat_list:
    rat_ss = df.loc[df['RatID']==rat,'Session'].unique()
    for ss in rat_ss:
        target = cell_finder(rat,ss)
        
        if len(target)==1:
            continue
        
        fig,ax = plt.subplots(len(target),1,figsize=(30,len(target)*2))
        for i,cell in enumerate(target):
            if cell < 10:
                cell_num = '000'+str(cell)
            elif cell < 100:
                cell_num = '00'+str(cell)
            elif cell < 1000:
                cell_num = '0'+str(cell)
            else:
                cell_num = str(cell)
            cell_full_path = glob(zfr_path+cell_num+'*')
            zfr = pd.read_csv(cell_full_path[0])
    
            # remove error trials and control trials
            zfr.drop(zfr[zfr['Correctness']==0].index,inplace=True)
            zfr.drop(zfr[zfr['Type']=='Elemental'].index,inplace=True)
            zfr.reset_index(inplace=True,drop=True)
    
            mfr = zfr.iloc[:,9:].mean(axis=1)
    
            ax[i].plot(mfr,color='black')
            ax[i].set_ylim([-1,2])
            ax[i].set_yticks([-1,0,1,2])
            ax[i].set_ylabel('zFR')
            ax[i].text(-2,0.5,df.loc[df['Key']==cell,'Cat'].values[0],fontsize=15)
    
            alpha_level = 0.4
            obj = 'Left'    # 'Left' or 'Right
        
            for j in range(len(zfr)):
                if (zfr.loc[j,'Type']=='Multimodal')&(zfr.loc[j,'RWD_Loc']==obj):
                    rect = patches.Rectangle((j-0.5,-1),1,4,facecolor='purple',alpha=alpha_level)
                elif (zfr.loc[j,'Type']=='Visual')&(zfr.loc[j,'RWD_Loc']==obj):
                    rect = patches.Rectangle((j-0.5,-1),1,4,facecolor='blue',alpha=alpha_level)
                elif (zfr.loc[j,'Type']=='Auditory')&(zfr.loc[j,'RWD_Loc']==obj):
                    rect = patches.Rectangle((j-0.5,-1),1,4,facecolor='red',alpha=alpha_level)
                else:
                    continue
                ax[i].add_patch(rect)            
        plt.savefig(fig_path+'ensemble/'+f'LE{rat} Day{ss} Object for {obj} Choice.png',dpi=100,facecolor='white')
        plt.close()

print('********END********')

********END********
CPU times: total: 51.7 s
Wall time: 59.2 s
